In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

## Ejemplo 1 - Conjunto de datos separables linealmente

Comenzaremos con un ejemplo de conjunto de datos en 2D que puede ser separado por un límite lineal.

In [ ]:
#Primero cargamos y graficamos los datos
data = np.load("ex1_linear.npz")

X = data['X']
y = data['y'][:,0]

pos = y == 1
neg = y == 0

plt.plot(X[pos,0], X[pos,1], 'X', c='blue')
plt.plot(X[neg,0], X[neg,1], 'o', c='red')

En general existen infinitas soluciones posibles para el problema de separar ambas clases:

In [ ]:
plt.plot(X[pos,0], X[pos,1], 'X', c='blue')
plt.plot(X[neg,0], X[neg,1], 'o', c='red')
xfit = np.linspace(0, 4)
for w, b in [(-0.6, 4.5), (-0.7, 5.0), (-1, 5.6)]:
  plt.plot(xfit, w * xfit + b, '-k')

In [ ]:
#Según modelo de regresión logistica
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(random_state=0).fit(X, y)

w_lr = model_lr.coef_[0]
b_lr = model_lr.intercept_
xp_lr = np.linspace(min(X[:, 0]), max(X[:, 0]), 100)
yp_lr = -(w_lr[0] * xp_lr + b_lr)/w_lr[1]

plt.plot(X[pos,0], X[pos,1], 'X', c='blue')
plt.plot(X[neg,0], X[neg,1], 'o', c='red')
plt.plot(xp_lr, yp_lr, '-k')

In [ ]:
#Entrenamos un clasificador SVM utilizando un kernel lineal

C = 1
model = svm.SVC(random_state=0, kernel="linear", C=C)
model.fit(X, y)

# Calculamos la función de decision
w = model.coef_[0]
b = model.intercept_
xp = np.linspace(min(X[:, 0]), max(X[:, 0]), 100)
yp = -(w[0] * xp + b)/w[1]

# Graficamos la funcioń de decision
plt.plot(X[pos,0], X[pos,1], 'X', c='blue')
plt.plot(X[neg,0], X[neg,1], 'o', c='red')
plt.plot(xp, yp, '-g', label='SVM')
plt.plot(xp_lr, yp_lr, '-y', label='LR')
plt.legend()

In [ ]:
# Calculamos los vectores de soporte

def plot_svc_decision_function(model, ax=None, plot_support=True):
  """Plot the decision function for a two-dimensional SVC"""
  if ax is None:
    ax = plt.gca()
  xlim = ax.get_xlim()
  ylim = ax.get_ylim()
  # create grid to evaluate model
  x = np.linspace(xlim[0], xlim[1], 30)
  y = np.linspace(ylim[0], ylim[1], 30)
  Y, X = np.meshgrid(y, x)
  xy = np.vstack([X.ravel(), Y.ravel()]).T
  P = model.decision_function(xy).reshape(X.shape)
  # plot decision boundary and margins
  ax.contour(X, Y, P, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
  # plot support vectors
  if plot_support:
    ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], s=300, linewidth=2, alpha=.5);
  ax.set_xlim(xlim)
  ax.set_ylim(ylim)

plt.plot(X[pos,0], X[pos,1], 'X', c='blue')
plt.plot(X[neg,0], X[neg,1], 'o', c='red')
plot_svc_decision_function(model);

## Kernel trick

In [ ]:
from sklearn import datasets

# Generar un set de datos no lineal
X, y = datasets.make_circles(n_samples=100, factor=0.6, noise=0.1)

# Graficar los datos
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

In [ ]:
import plotly.express as px

X, y = datasets.make_circles(n_samples=100, factor=0.6, noise=0.1)

# Definir los parametros de la función gaussiana
gamma = 0.5

# Crear una grilla de puntos graficar la función gaussiana
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),np.arange(y_min, y_max, 0.05))
zz = np.exp(-(xx.ravel()**2 + yy.ravel()**2)/(2*gamma**2))


# Compute the Gaussian function values
Z = np.exp(-(X[:, 0].ravel()**2 + X[:, 1].ravel()**2)/(2*gamma**2))

fig1 = px.scatter_3d(x=X[:, 0], y=X[:, 1], z=np.zeros(X.shape[0]), color=y)
fig2 = px.scatter_3d(x=xx.ravel(), y=yy.ravel(), z=zz, opacity=0.1, color_discrete_sequence=["gray"])

fig1.add_traces(
    list(fig2.select_traces()))

# Show the figure
fig1.show()

In [ ]:
# Compute the Gaussian function values
Z = np.exp(-(X[:, 0].ravel()**2 + X[:, 1].ravel()**2)/(2*gamma**2))

fig1 = px.scatter_3d(x=X[:, 0], y=X[:, 1], z=Z, color=y)
fig2 = px.scatter_3d(x=xx.ravel(), y=yy.ravel(), z=zz, opacity=0.1, color_discrete_sequence=["gray"])

fig1.add_traces(
    list(fig2.select_traces()))

# Show the figure
fig1.show()


#### Separación Lineal en el Nuevo Espacio

In [ ]:
# Agregamos una nueva característica a la matriz X, correspondiente a la nueva dimension
newX = np.hstack((X,Z.reshape(-1,1)))

# Entrenamos un modelo de SVM lineal en el nuevo espacio de caracteristicas
svm_model_3d_linear = svm.SVC(kernel='linear', C=100)
svm_model_3d_linear.fit(newX, y)

# Coeficientes del plano de decisión
coeficientes = svm_model_3d_linear.coef_
intercepto = svm_model_3d_linear.intercept_

# Calcular el valor z sobre plano de decisión
z = (-coeficientes[:,0] * xx.ravel() - coeficientes[:,1] * yy.ravel() - intercepto) / coeficientes[:,2]

# graficamos los datos
fig1 = px.scatter_3d(x=X[:, 0], y=X[:, 1], z=Z, color=y, color_discrete_sequence={0:"red", 1:"blue"})

# Crear una figura 2D (scatter plot) para visualizar el plano de decisión
fig2 = px.scatter_3d(x=xx.ravel(), y=yy.ravel(), z=z)

fig1.add_traces(
    list(fig2.select_traces()))

# Mostrar la figura
fig1.show()


### Volviendo al espacio original

In [ ]:
# Entrenar un modelo de SVM con los mismos datos pero usando un kernel gaussiano o RBF
svm_model = svm.SVC(kernel='rbf', C=1, gamma='scale')
svm_model.fit(X, y)

# Creamos  a mesh grid to plot the decision boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Plot the decision boundary
Z = svm_model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contour(xx, yy, Z, alpha=0.5)
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title('SVM con kernel RBF')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()